# Workshop : รถขับเคลื่อนอัตโนมัติด้วยปัญญาประดิษฐ์ (AI)
## Chiang Mai Maker Party 07-08 December 2019

## Example 3 : Object Detection

### การทดสอบโมเดล (Evaluate)


### เรียกใช้งานไลบรารี่

In [ ]:
import argparse
import json
import cv2
import numpy as np
from yolo.frontend import create_yolo
from yolo.backend.utils.box import draw_scaled_boxes
from yolo.backend.utils.annotation import parse_annotation
from yolo.backend.utils.eval.fscore import count_true_positives, calc_score

from pascal_voc_writer import Writer
from shutil import copyfile
import os
import yolo

### เรียกใช้งาน Artificial Neural Network(ANN) 

In [ ]:
def create_ann(filename, image, boxes, labels,label_list):
    copyfile(os.path.join('tests/dataset/svhn/numbers_pics',filename), 'imgs/'+filename)
    writer = Writer(os.path.join('tests/dataset/svhn/numbers_pics',filename), 224, 224)
    writer.addObject(label_list[labels[0]], boxes[0][0], boxes[0][1], boxes[0][2], boxes[0][3])
    name = filename.split('.')
    writer.save('ann/'+name[0]+'.xml')

### โหลดไฟล์ config.json และ Model

โหลดโมเดลด้วยคำสั่ง yolo.load_weights("model.h5")

In [ ]:
from yolo.frontend import create_yolo
from yolo.backend.utils.box import draw_scaled_boxes
from yolo.backend.utils.annotation import parse_annotation
from yolo.backend.utils.eval.fscore import count_true_positives, calc_score
import yolo.backend.utils.box as box

with open("data/config.json") as config_buffer:
    config = json.loads(config_buffer.read())
    
yolo = create_yolo(config['model']['architecture'],
                   config['model']['labels'],
                   config['model']['input_size'],
                   config['model']['anchors'])
yolo.load_weights("jetbot_demo.h5")

In [ ]:
import traitlets
import ipywidgets
import ipywidgets.widgets as widgets
from IPython.display import display
import numpy as np
import cv2

# init jetcam camera
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=30)
camera.running = True


### เปิดกล้องจากหุ่นยนต์ Jetbot

In [ ]:
image = camera.value
image_widget = ipywidgets.Image(format='jpeg')
label_widget = ipywidgets.Label()
image_widget.value = bgr8_to_jpeg(image)

display(image_widget, label_widget)

### รันโปรแกรมเพื่อ predict และแสดงผล

In [ ]:
camera.running = True
try:
    while 1 :
        image = camera.value

        boxes, probs = yolo.predict(image, 0.7)
        image = box.draw_boxes(image, boxes, probs, config['model']['labels'])
        image_widget.value = bgr8_to_jpeg(image)
        
except KeyboardInterrupt:
    camera.running = False